# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import utils
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures

# Import Datasets

In [2]:
games = pd.read_csv('../input/nba-games/games.csv')
details = pd.read_csv('../input/nba-games/games_details.csv')
teams = pd.read_csv('../input/nba-games/teams.csv')
players = pd.read_csv('../input/nba-games/players.csv')
ranking = pd.read_csv('../input/nba-games/ranking.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Data Preparation

## Get Labels
Here we drop the rows when a team didn't play enough games in one season (not 82 games).

In [3]:
def get_labels(ranking):
    temp = ranking.copy(deep=False)
    temp = temp.groupby(['TEAM_ID','SEASON_ID'])['G','W'].max()
    temp = pd.DataFrame(temp)
    temp.reset_index(inplace=True)
    drops = []
    for i in range(len(temp)):
        if temp.iloc[i,1] / 10000 > 2:
            temp.iloc[i,1] = temp.iloc[i,1] % 10000
        else:
            drops.append(i)
            continue;
        if (temp.iloc[i,2] != 82):
            drops.append(i)
    for i in range(len(drops)):
        temp.drop([drops[i]], inplace=True)
    temp.reset_index(inplace=True)
    temp.drop(columns=['index'], inplace=True)
    temp.drop(columns=['G'], inplace=True)
#     temp = pd.merge(temp, ranking, how='left', left_on=['TEAM_ID','STANDINGSDATE'], right_on = ['TEAM_ID','STANDINGSDATE'])
#     temp.drop(columns=['STANDINGSDATE','LEAGUE_ID','SEASON_ID_y','CONFERENCE','TEAM','G','W','L','HOME_RECORD','ROAD_RECORD','RETURNTOPLAY'], inplace=True)
    return temp

In [4]:
labels = get_labels(ranking)
labels

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TEAM_ID,SEASON_ID,W
0,1610612737,2002,35
1,1610612737,2003,28
2,1610612737,2004,13
3,1610612737,2005,26
4,1610612737,2006,30
...,...,...,...
501,1610612766,2015,48
502,1610612766,2016,36
503,1610612766,2017,36
504,1610612766,2018,39


## Get Features

In [5]:
def get_features(games, details):
    temp = pd.merge(games, details, how='left', left_on=['GAME_ID'], right_on = ['GAME_ID'])
    temp = temp[['TEAM_ID','SEASON','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS']]
    temp = temp.groupby(['TEAM_ID','SEASON'])['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS'].sum()
    temp = pd.DataFrame(temp)
    next_season = []
    temp.reset_index(inplace=True)
    for i in range(len(temp)):
        next_season.append(temp.iloc[i,1] + 1)
    temp['NEXT_SEASON'] = next_season
    return temp

In [6]:
features = get_features(games, details)
features

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,TEAM_ID,SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_SEASON
0,1.610613e+09,2003,2857.0,6609.0,320.764,422.0,1256.0,104.083,1555.0,2004.0,...,2543.0,3548.0,1666.0,633.0,410.0,1292.0,1853.0,7691.0,-1905.0,2004
1,1.610613e+09,2004,2997.0,6821.0,357.705,309.0,997.0,114.160,1456.0,2059.0,...,2387.0,3510.0,1649.0,642.0,350.0,1289.0,2068.0,7759.0,-3983.0,2005
2,1.610613e+09,2005,3196.0,6997.0,358.522,450.0,1205.0,138.741,1804.0,2404.0,...,2427.0,3568.0,1759.0,633.0,417.0,1377.0,2235.0,8646.0,-1950.0,2006
3,1.610613e+09,2006,3105.0,6986.0,369.110,385.0,1161.0,127.166,1865.0,2438.0,...,2528.0,3595.0,1728.0,663.0,480.0,1400.0,2195.0,8460.0,-1950.0,2007
4,1.610613e+09,2007,3457.0,7668.0,382.692,436.0,1250.0,122.879,2129.0,2758.0,...,2885.0,4074.0,2084.0,696.0,536.0,1411.0,2016.0,9479.0,-1165.0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,1.610613e+09,2018,3495.0,7788.0,413.804,1043.0,2949.0,245.989,1619.0,2046.0,...,2980.0,3824.0,2029.0,625.0,435.0,1038.0,1679.0,9652.0,-330.0,2019
595,1.610613e+09,2019,2615.0,6015.0,297.664,839.0,2397.0,180.742,1135.0,1506.0,...,2249.0,3012.0,1668.0,466.0,297.0,1008.0,1333.0,7204.0,-2260.0,2020
596,1.610613e+09,2020,3210.0,7117.0,352.732,1095.0,2976.0,211.146,1299.0,1729.0,...,2704.0,3559.0,2173.0,640.0,386.0,1160.0,1485.0,8814.0,-899.0,2021
597,1.610613e+09,2021,3685.0,7945.0,398.470,1190.0,3301.0,218.024,1373.0,1859.0,...,2934.0,3877.0,2409.0,747.0,428.0,1129.0,1736.0,9933.0,-380.0,2022


## Get Full Dataset
Here we drop NaN values. We then do inner join of the two tables, so some values are also dropped because some values don't exist in both tables.

In [7]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [8]:
def get_data(ranking, games, details):
    labels = get_labels(ranking)
    features = get_features(games, details)
    data = pd.merge(labels, features, how='left', left_on=['TEAM_ID','SEASON_ID'], right_on = ['TEAM_ID','NEXT_SEASON'])
    data.drop(columns=['SEASON_ID','SEASON'], inplace=True)
    data.dropna(inplace=True)
    data = swap_columns(data, 'W', 'NEXT_SEASON')
    data = data.astype({'NEXT_SEASON': 'int64'})
    data.rename(columns={'W' : 'NEXT_W'}, inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=['index'], inplace=True)
    return data

In [9]:
data = get_data(ranking, games, details)
data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,TEAM_ID,NEXT_SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_W
0,1610612737,2004,2857.0,6609.0,320.764,422.0,1256.0,104.083,1555.0,2004.0,...,2543.0,3548.0,1666.0,633.0,410.0,1292.0,1853.0,7691.0,-1905.0,13
1,1610612737,2005,2997.0,6821.0,357.705,309.0,997.0,114.160,1456.0,2059.0,...,2387.0,3510.0,1649.0,642.0,350.0,1289.0,2068.0,7759.0,-3983.0,26
2,1610612737,2006,3196.0,6997.0,358.522,450.0,1205.0,138.741,1804.0,2404.0,...,2427.0,3568.0,1759.0,633.0,417.0,1377.0,2235.0,8646.0,-1950.0,30
3,1610612737,2007,3105.0,6986.0,369.110,385.0,1161.0,127.166,1865.0,2438.0,...,2528.0,3595.0,1728.0,663.0,480.0,1400.0,2195.0,8460.0,-1950.0,37
4,1610612737,2008,3457.0,7668.0,382.692,436.0,1250.0,122.879,2129.0,2758.0,...,2885.0,4074.0,2084.0,696.0,536.0,1411.0,2016.0,9479.0,-1165.0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,1610612766,2015,3191.0,7612.0,370.551,551.0,1736.0,154.944,1547.0,2057.0,...,3070.0,3965.0,1833.0,552.0,492.0,1017.0,1691.0,8480.0,-1430.0,48
443,1610612766,2016,3528.0,8122.0,425.323,987.0,2745.0,266.812,1859.0,2352.0,...,3403.0,4246.0,2027.0,691.0,485.0,1140.0,1758.0,9902.0,1195.0,36
444,1610612766,2017,3318.0,7589.0,387.285,866.0,2526.0,188.367,1741.0,2146.0,...,3111.0,3904.0,2024.0,621.0,425.0,1004.0,1486.0,9243.0,-175.0,36
445,1610612766,2018,3386.0,7537.0,390.534,872.0,2379.0,212.096,1738.0,2334.0,...,3078.0,3954.0,1869.0,594.0,393.0,1093.0,1502.0,9382.0,90.0,39


In [10]:
data.to_csv('/kaggle/working/nba_data.csv')

## Feature Scaling
We use z-score normalization.

In [11]:
def scale_data(data):
    temp = data.copy(deep=False)
    std_slc = StandardScaler()
    preprocess = std_slc.fit_transform(temp[['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS']])
#     preprocess = preprocessing.normalize(temp[['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
#              ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS']])
    data_scaled = pd.DataFrame(preprocess, columns=['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM'
                    ,'FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS'])
    data_scaled.insert(0,'TEAM_ID',temp[['TEAM_ID']])
    data_scaled.insert(1,'NEXT_SEASON',temp[['NEXT_SEASON']])
    data_scaled.insert(21,'NEXT_W',temp[['NEXT_W']])
    return data_scaled

In [12]:
data_scaled = scale_data(data)
data_scaled

,TEAM_ID,NEXT_SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_W
0,1610612737,2004,-1.411461,-1.264087,-1.498915,-1.106395,-1.073258,-1.705786,-0.471266,-0.614104,...,-0.976307,-0.816891,-1.158351,-0.629734,-0.391299,0.083757,-0.344346,-1.358318,-0.988451,13
1,1610612737,2005,-1.059369,-0.977735,-0.786667,-1.577773,-1.486932,-1.490412,-0.839727,-0.461249,...,-1.413032,-0.907023,-1.218349,-0.541993,-1.073390,0.063306,0.511068,-1.297323,-2.069663,26
2,1610612737,2006,-0.558896,-0.740009,-0.770914,-0.989593,-1.154715,-0.965047,0.455471,0.497565,...,-1.301051,-0.769454,-0.830124,-0.629734,-0.311722,0.663208,1.175506,-0.501689,-1.011865,30
3,1610612737,2007,-0.787756,-0.754867,-0.566770,-1.260740,-1.224992,-1.212437,0.682503,0.592057,...,-1.018299,-0.705412,-0.939533,-0.337266,0.404473,0.820000,1.016359,-0.668530,-1.011865,37
4,1610612737,2008,0.097503,0.166323,-0.304900,-1.047994,-1.082842,-1.304063,1.665067,1.481392,...,-0.018870,0.430724,0.316904,-0.015550,1.041091,0.894988,0.304177,0.245508,-0.603419,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,1610612766,2015,-0.571471,0.090682,-0.538987,-0.568274,-0.306604,-0.618743,-0.501040,-0.466808,...,0.499041,0.172188,-0.568954,-1.419399,0.540891,-1.790936,-0.988891,-0.650590,-0.741302,48
443,1610612766,2016,0.276064,0.779548,0.517056,1.250493,1.304966,1.772191,0.660172,0.353048,...,1.431282,0.838690,0.115733,-0.064295,0.461314,-0.952437,-0.722320,0.624936,0.624522,36
444,1610612766,2017,-0.252074,0.059616,-0.216343,0.745744,0.955180,0.095600,0.220995,-0.219461,...,0.613822,0.027502,0.105145,-0.746721,-0.220777,-1.879558,-1.804519,0.033817,-0.088308,36
445,1610612766,2018,-0.081058,-0.010621,-0.153700,0.770773,0.720392,0.602756,0.209830,0.303023,...,0.521438,0.146097,-0.441899,-1.009943,-0.584558,-1.272839,-1.740860,0.158499,0.049575,39


In [13]:
data_scaled.to_csv('/kaggle/working/nba_data_scaled.csv')

# Training

## Feature/Label Split

In [14]:
def split_data_X_y(data):
    temp = data.copy(deep=False)
    temp.drop(columns=['TEAM_ID','NEXT_SEASON'], inplace=True)
    X = data.iloc[:,2:].copy(deep=False)
    X.drop(columns=['NEXT_W'], inplace=True)
    y = data.iloc[:,-1:].copy(deep=False)
    return X, y

## Train/Val Split

In [15]:
def split_data_train_test(data):
    temp = data.copy(deep=False)
    temp.drop(columns=['TEAM_ID','NEXT_SEASON'], inplace=True)
    X = data.iloc[:,2:].copy(deep=False)
    X.drop(columns=['NEXT_W'], inplace=True)
    y = data.iloc[:,-1:].copy(deep=False)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023)
    return X_train, X_test, y_train, y_test

In [16]:
def rmse(y_test, y_pred):
    return mean_squared_error(y_test, y_pred, squared=False)

In [17]:
X, y = split_data_X_y(data_scaled)

In [18]:
X_train, X_test, y_train, y_test = split_data_train_test(data_scaled)

## Cross Validation and K-Folds

In [19]:
def cross_validation(model, _X, _y, _cv=5):
      '''Function to perform 5 Folds Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      _scoring = make_scorer(rmse, greater_is_better=True)
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {"Training RMSE": results['train_score'],
              "Mean RMSE": results['train_score'].mean(),
              "Validation RMSE": results['test_score'],
              "Mean Validation RMSE": results['test_score'].mean()
              }

## Linear Regression

In [20]:
model_lr = LinearRegression()
print(cross_validation(model_lr, X, y))

{'Training RMSE': array([9.18583535, 9.62878426, 9.4436024 , 9.45689719, 9.79154861]), 'Mean RMSE': 9.501333562167394, 'Validation RMSE': array([11.28546485,  9.63820163, 10.42139581, 10.45118549,  8.74554251]), 'Mean Validation RMSE': 10.108358057967077}


## Lasso Regression

In [21]:
model_lasso = linear_model.Lasso(alpha=0.1)
print(cross_validation(model_lasso, X, y))

{'Training RMSE': array([9.23009246, 9.67631003, 9.50594824, 9.51493077, 9.83293072]), 'Mean RMSE': 9.552042446379847, 'Validation RMSE': array([11.11171298,  9.50638423, 10.14952278, 10.08427602,  8.69314068]), 'Mean Validation RMSE': 9.909007336211205}


## Support Vector Machine Regression

In [22]:
model_svm = SVR(kernel = 'linear').fit(X_train, y_train)
print(cross_validation(model_svm, X, y))

{'Training RMSE': array([9.3048602 , 9.72516725, 9.5384157 , 9.63658177, 9.88048905]), 'Mean RMSE': 9.617102792719127, 'Validation RMSE': array([11.53380708,  9.86140631, 10.21292427, 10.83903829,  8.79618425]), 'Mean Validation RMSE': 10.248672042362129}


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

## Random Forest Regression

In [23]:
model_tree = RandomForestRegressor(random_state=2023)
print(cross_validation(model_tree, X, y))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 

{'Training RMSE': array([3.68165906, 3.98833593, 3.87518098, 3.88696644, 3.97655774]), 'Mean RMSE': 3.8817400316406085, 'Validation RMSE': array([11.80599288, 10.20663352, 10.62018912, 10.45245619,  8.94105146]), 'Mean Validation RMSE': 10.40526463139284}


# Feature Selection

## Lasso Regression Coefficient

In [24]:
model_lasso = linear_model.Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)
y_pred = model_lasso.predict(X_test)
print(mean_squared_error(y_test, y_pred, squared=False))
print(model_lasso.coef_)

10.10864911978995
[ 0.          0.         -0.5065314   0.          0.         -0.66973622
  0.         -0.         -0.19109535  0.          0.72932052  0.17580654
 -0.          0.94379254  0.          0.4820294   0.01080316  0.
  7.50363662]


In [25]:
def feature_selection(data):
    temp = data.copy(deep=False)
    temp.drop(columns=['FGM','FGA','FG3M','FG3A','FTM','FTA','OREB','AST','BLK','PTS'], inplace=True)
    return temp;

In [26]:
data_selected = feature_selection(data_scaled)
X_new, y_new = split_data_X_y(data_selected)
X_new_train, X_new_test, y_new_train, y_new_test = split_data_train_test(data_selected)
data_selected

,TEAM_ID,NEXT_SEASON,FG_PCT,FG3_PCT,FT_PCT,DREB,REB,STL,TO,PF,PLUS_MINUS,NEXT_W
0,1610612737,2004,-1.498915,-1.705786,-0.861508,-0.976307,-0.816891,-0.629734,0.083757,-0.344346,-0.988451,13
1,1610612737,2005,-0.786667,-1.490412,-0.701850,-1.413032,-0.907023,-0.541993,0.063306,0.511068,-2.069663,26
2,1610612737,2006,-0.770914,-0.965047,0.197828,-1.301051,-0.769454,-0.629734,0.663208,1.175506,-1.011865,30
3,1610612737,2007,-0.566770,-1.212437,0.186456,-1.018299,-0.705412,-0.337266,0.820000,1.016359,-1.011865,37
4,1610612737,2008,-0.304900,-1.304063,1.040891,-0.018870,0.430724,-0.015550,0.894988,0.304177,-0.603419,47
...,...,...,...,...,...,...,...,...,...,...,...,...
442,1610612766,2015,-0.538987,-0.618743,0.104340,0.499041,0.172188,-1.419399,-1.790936,-0.988891,-0.741302,48
443,1610612766,2016,0.517056,1.772191,1.550633,1.431282,0.838690,-0.064295,-0.952437,-0.722320,0.624522,36
444,1610612766,2017,-0.216343,0.095600,1.398407,0.613822,0.027502,-0.746721,-1.879558,-1.804519,-0.088308,36
445,1610612766,2018,-0.153700,0.602756,0.632825,0.521438,0.146097,-1.009943,-1.272839,-1.740860,0.049575,39


# Retraining

## Linear Regression

In [27]:
model_lr_new = LinearRegression()
print(cross_validation(model_lr_new, X_new, y_new))

{'Training RMSE': array([9.22851245, 9.71357778, 9.53587644, 9.53971929, 9.84449558]), 'Mean RMSE': 9.572436305916387, 'Validation RMSE': array([11.28666531,  9.32700823, 10.06012399, 10.09426453,  8.68503644]), 'Mean Validation RMSE': 9.890619698782485}


## Lasso Regression

In [28]:
model_lasso_new = linear_model.Lasso(alpha=0.1)
print(cross_validation(model_lasso_new, X_new, y_new))

{'Training RMSE': array([9.2407124 , 9.72543766, 9.5492515 , 9.55080027, 9.85387941]), 'Mean RMSE': 9.584016249349592, 'Validation RMSE': array([11.10286677,  9.3476289 , 10.00699856,  9.99603348,  8.7137333 ]), 'Mean Validation RMSE': 9.833452201944691}


## Support Vector Machine Regression

In [29]:
model_svm_new = SVR(kernel = 'linear')
print(cross_validation(model_svm_new, X_new, y_new))

{'Training RMSE': array([9.25062681, 9.7462161 , 9.59932656, 9.57569686, 9.89479155]), 'Mean RMSE': 9.613331575118089, 'Validation RMSE': array([11.26103375,  9.35133074, 10.27951509, 10.12513306,  8.65664761]), 'Mean Validation RMSE': 9.934732049648634}


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

## Random Forest Regression

In [30]:
model_tree_new = RandomForestRegressor(random_state=2023)
print(cross_validation(model_tree_new, X_new, y_new))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 

{'Training RMSE': array([3.6610285 , 3.96493372, 3.89475274, 3.89762379, 3.94863501]), 'Mean RMSE': 3.873394753061979, 'Validation RMSE': array([11.83831257,  9.89979781, 10.75042539, 10.37173899,  8.95092118]), 'Mean Validation RMSE': 10.362239186319268}


# Testing Demo

In our demo, we just use the Lasso Regression Model to predict the number of game wins in 2018, using team statistics from 2017. This is because data in later years are somewhat incomplete.

In [31]:
model_lasso_new = linear_model.Lasso(alpha=0.1)
model_lasso_new.fit(X_new_train, y_new_train)

Lasso(alpha=0.1)

In [32]:
def get_2017(data):
    temp = data.copy(deep=False)
    drop_list = []
    for i in range(len(temp)):
        if temp.iloc[i, 1] != 2018:
            drop_list.append(i)
            
    temp.drop(drop_list, inplace=True)
    temp.reset_index(inplace=True)
    temp.drop(columns=['index'], inplace=True)
    return temp

In [33]:
data_2017 = get_2017(data_scaled)
data_2017

,TEAM_ID,NEXT_SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_W
0,1610612737,2018,-0.304887,-0.161902,0.294229,1.171235,1.233092,1.818036,-1.074203,-1.239417,...,-0.083259,-0.586818,0.228671,-0.191031,-0.925604,0.288269,-0.965019,-0.224517,-1.196576,29
1,1610612738,2018,1.432937,1.854719,0.992268,2.088961,2.026899,2.574612,0.127949,0.022327,...,2.119965,1.744752,1.213351,0.744868,0.302160,0.479147,0.602577,1.502197,0.918499,49
2,1610612739,2018,2.232688,2.192399,2.213913,2.364279,2.427795,2.263659,0.983971,0.828286,...,2.069573,1.638017,1.640399,0.559638,-0.186672,1.201756,0.487196,2.338195,0.013153,19
3,1610612740,2018,1.523474,1.113175,0.255263,1.137863,1.178788,1.281108,-0.515928,-0.641895,...,1.400487,0.734326,1.848628,0.627880,1.222982,0.676842,-0.427899,1.207085,0.138029,33
4,1610612741,2018,-0.065968,0.327058,-0.177917,1.246322,1.319341,1.071526,-1.528267,-1.495101,...,0.535435,0.063081,0.316904,-0.230027,-1.414435,-0.563864,-0.833723,-0.147376,-1.615429,22
5,1610612742,2018,-0.164051,0.013691,0.290469,1.521640,1.573295,1.361726,-1.613869,-1.592372,...,0.132304,-0.551239,-0.000734,-0.619985,-1.243913,-1.504620,-0.996849,-0.178771,-0.572200,33
6,1610612743,2018,0.271034,-0.016024,-0.178862,1.308894,1.205940,1.278415,-0.690854,-0.733608,...,0.062316,-0.067373,0.669836,-0.259274,-0.243513,-0.134389,-1.207718,0.308297,0.312334,54
7,1610612744,2018,2.841303,2.158631,3.822270,2.172391,1.959816,0.771195,0.347538,-0.105515,...,2.413914,1.891810,3.757989,1.554030,3.587563,1.876645,0.702044,2.577693,1.878479,57
8,1610612745,2018,1.322279,1.527846,0.435229,3.636582,3.862077,1.299894,1.069573,0.831066,...,1.814817,1.343903,0.641602,1.690515,0.665941,0.438244,0.272348,1.982986,1.974737,53
9,1610612746,2018,0.157862,-0.202423,0.454625,0.566370,0.630950,0.049414,-0.017202,0.097364,...,0.115507,-0.221546,-0.156024,-0.200780,-0.675504,-0.141206,-0.706406,0.230259,-0.020667,48


In [34]:
data_2017_selected = feature_selection(data_2017)
X_2017, y_2018 = split_data_X_y(data_2017_selected)

In [35]:
y_2018_pred = model_lasso_new.predict(X_2017)
print(np.round(y_2018_pred))

[31. 48. 41. 44. 28. 35. 43. 57. 58. 41. 39. 40. 40. 43. 34. 33. 32. 45.
 51. 25. 45. 28. 45. 47. 52. 50. 31. 42. 40. 40.]


In [36]:
team_id_name = {}
for i in range(len(teams)):
    team_id_name[teams.iloc[i, 1]] = teams.iloc[i, 5]
print(team_id_name)

{1610612737: 'Hawks', 1610612738: 'Celtics', 1610612740: 'Pelicans', 1610612741: 'Bulls', 1610612742: 'Mavericks', 1610612743: 'Nuggets', 1610612745: 'Rockets', 1610612746: 'Clippers', 1610612747: 'Lakers', 1610612748: 'Heat', 1610612749: 'Bucks', 1610612750: 'Timberwolves', 1610612751: 'Nets', 1610612752: 'Knicks', 1610612753: 'Magic', 1610612754: 'Pacers', 1610612755: '76ers', 1610612756: 'Suns', 1610612757: 'Trail Blazers', 1610612758: 'Kings', 1610612759: 'Spurs', 1610612760: 'Thunder', 1610612761: 'Raptors', 1610612762: 'Jazz', 1610612763: 'Grizzlies', 1610612764: 'Wizards', 1610612765: 'Pistons', 1610612766: 'Hornets', 1610612739: 'Cavaliers', 1610612744: 'Warriors'}


In [37]:
name_list = []
for i in range(len(data_2017)):
    name_list.append(team_id_name[data_2017.iloc[i, 0]])
print(name_list)

['Hawks', 'Celtics', 'Cavaliers', 'Pelicans', 'Bulls', 'Mavericks', 'Nuggets', 'Warriors', 'Rockets', 'Clippers', 'Lakers', 'Heat', 'Bucks', 'Timberwolves', 'Nets', 'Knicks', 'Magic', 'Pacers', '76ers', 'Suns', 'Trail Blazers', 'Kings', 'Spurs', 'Thunder', 'Raptors', 'Jazz', 'Grizzlies', 'Wizards', 'Pistons', 'Hornets']


In [38]:
prediction_dict_2018 = {"team_name": name_list, "wins_pred_2018": np.round(y_2018_pred), 'wins_2018': y_2018['NEXT_W']}
prediction_df_2018 = pd.DataFrame(prediction_dict_2018)
prediction_df_2018.sort_values(by='wins_pred_2018', ascending=False, inplace=True)
prediction_df_2018 = prediction_df_2018.astype({'wins_pred_2018': 'int64'})
prediction_df_2018.reset_index(inplace=True)
prediction_df_2018.drop(columns=['index'], inplace=True)

In [39]:
prediction_df_2018

,team_name,wins_pred_2018,wins_2018
0,Rockets,58,53
1,Warriors,57,57
2,Raptors,52,58
3,76ers,51,51
4,Jazz,50,50
5,Celtics,48,49
6,Thunder,47,49
7,Spurs,45,48
8,Trail Blazers,45,53
9,Pacers,45,48


In [40]:
prediction_df_2018.to_csv('/kaggle/working/nba_data_2018_prediction.csv')

# Acknowledgement
This project is finished under the mentorship of [Andrea Boskovic](https://stat.uw.edu/about-us/people/andrea-boskovic) and [Harshil Desai](https://stat.uw.edu/about-us/people/harshil-desai), Ph.D. students in Statistics at the University of Washington.